In [1]:
import pandas as pd
import numpy as np
import re
from re import sub
import multiprocessing

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


from time import time 
from collections import defaultdict

In [2]:
df=pd.read_csv("../../../jigsaw-toxic-severity-rating/validation_data.csv")

In [3]:
df

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"
...,...,...,...
30103,461,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30104,527,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30105,352,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook
30106,311,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook


In [4]:
from nltk.corpus import stopwords
import re
def text_cleaning(text):
    if text:
        text = ' '.join(text.split('.'))
        text = re.sub('\/',' ',text)
        text = re.sub(r'\\',' ',text)
        text = re.sub(r'((http)\S+)','',text)
        text = re.sub(r'\s+', ' ', re.sub('[^A-Za-z]', ' ', text.strip().lower())).strip()
        text = re.sub(r'\W+', ' ', text.strip().lower()).strip()
        text = [word for word in text.split()]
        return text
    return []

In [5]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [6]:
def lemmatizing(s):
    l=s.split(" ")
    l2=[lemmatizer.lemmatize(word) for word in l]
    s=" ".join(l2)
    return s


In [69]:
df.less_toxic = df.less_toxic.apply(lambda x: ' '.join(text_cleaning(x)))

In [70]:
df.less_toxic = df.less_toxic.apply(lambda x: lemmatizing(x))


In [9]:
df

,worker,less_toxic,more_toxic
0,313,this article suck woo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,and yes people should recognize that but they ...,Daphne Guinness \n\nTop of the mornin' my fav...
2,82,western medium yup because every crime in the ...,"""Atom you don't believe actual photos of mastu..."
3,347,and you removed it you numbskull i don t care ...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina bluerasberry why don t you be a ...,"hey \n\nway to support nazis, you racist"
...,...,...,...
30103,461,i m sorry i m not an admin i will give you thr...,get out my large penis
30104,527,i m sorry i m not an admin i will give you thr...,get out my large penis
30105,352,wow are you out of your mind how wa my edit on...,Piss off you slant eyed-gook
30106,311,wow are you out of your mind how wa my edit on...,Piss off you slant eyed-gook


In [71]:
sentences = [word_tokenize(sent) for sent in df.less_toxic]

In [72]:
phrases = Phrases(sentences, min_count=30, progress_per=10000)

In [73]:
bigram = Phraser(phrases)

In [74]:
sentences2 = bigram[sentences]

In [75]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

w2v_model.build_vocab(sentences2, progress_per=50000)

In [76]:
w2v_model.train(sentences2, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(15971092, 64393050)

In [77]:
w2v_model.init_sims(replace=True)

C:\Users\Hector\AppData\Local\Temp/ipykernel_11228/514372312.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [78]:
w2v_model.save("word2vec.model")

In [26]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

In [79]:
word_vectors = Word2Vec.load("word2vec.model").wv

In [80]:
word_vectors

In [81]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

In [82]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)

[('burp', 0.9661408066749573),
 ('spel', 0.9638698101043701),
 ('kant', 0.9608436226844788),
 ('bithces', 0.9570140242576599),
 ('fainted', 0.9564516544342041),
 ('thingggggggggggg', 0.9560933709144592),
 ('ik', 0.9541478753089905),
 ('charna', 0.9535954594612122),
 ('marissa', 0.952586829662323),
 ('wierddd', 0.951858401298523)]

In [83]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

[('fourohfour', 0.9476852416992188),
 ('bail_out', 0.9469375610351562),
 ('hate_wikipedia', 0.9424804449081421),
 ('flabbergastingly', 0.9377520084381104),
 ('cuntt', 0.9334406852722168),
 ('vomit_vomit', 0.9327620267868042),
 ('nikko_smell', 0.932682454586029),
 ('jog', 0.9326014518737793),
 ('picture_showing', 0.932592511177063),
 ('making_love', 0.9322660565376282)]

In [84]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [85]:
words = pd.DataFrame(word_vectors.index_to_key)


In [86]:
words

,0
0,the
1,a
2,to
3,you
4,i
...,...
25083,maliyadeva
25084,massless
25085,matara
25086,suares


In [87]:
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [88]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [89]:
words

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,the,"[-0.04089157, -0.10525032, 0.06359307, 0.03510...",1,1,1.030860,1.030860
1,a,"[-0.016692968, -0.13745366, 0.027092986, -0.02...",1,1,1.046803,1.046803
2,to,"[-0.009316119, -0.12273791, 0.112017654, 0.009...",1,1,1.070928,1.070928
3,you,"[0.01881397, 0.05734981, 0.17161769, 0.0083904...",1,1,1.090952,1.090952
4,i,"[-0.0049191266, 0.0117665185, 0.13627413, -0.0...",1,1,1.099641,1.099641
...,...,...,...,...,...,...
25083,maliyadeva,"[0.025441179, 0.050027493, -0.016706299, 0.047...",0,-1,1.584844,-1.584844
25084,massless,"[0.024190784, 0.006043821, -0.05227672, 0.0457...",0,-1,1.566481,-1.566481
25085,matara,"[0.022312755, 0.05083658, -0.019353496, 0.0489...",0,-1,1.572785,-1.572785
25086,suares,"[-0.043216027, -0.010845546, -0.026594374, 0.0...",0,-1,1.630726,-1.630726


In [90]:
sentiment_map = words
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from IPython.display import display

In [91]:
sentences3=[]
for i in sentences2:
    sentences3.append(" ".join(i))

In [92]:
df["less_toxic"]=sentences3

In [93]:
df.less_toxic

0                       this article suck woo woo wooooooo
1        and yes people should recognize that but they ...
2        western medium yup because every crime in the ...
3        and you removed it you numbskull i don_t care ...
4        smelly vagina bluerasberry why don_t you be a ...
                               ...                        
30103    i m_sorry i m not an_admin i will give you thr...
30104    i m_sorry i m not an_admin i will give you thr...
30105    wow are you out of your mind how wa my edit on...
30106    wow are you out of your mind how wa my edit on...
30107    wow are you out of your mind how wa my edit on...
Name: less_toxic, Length: 30108, dtype: object

In [94]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x.split(), norm=None)
tfidf.fit(df.less_toxic)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(df.less_toxic)

C:\Users\Hector\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Hector\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [218]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this wonderful article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer

    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.text.split()))

In [102]:
%%time
replaced_tfidf_scores = df.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)#this step takes around 3-4 minutes minutes to calculate

KeyError: 'pus'

In [131]:
replaced_tfidf_scores

0        [2.0897177568367873, 2.776760753827239, 5.1534...
1        [1.6329856895354273, 4.745906979563401, 3.0650...
2        [6.363819518988473, 5.823641683209955, 8.47936...
3        [1.6329856895354273, 4.619652441241611, 4.6279...
4        [8.604529208264431, 14.071826580701172, 9.5208...
                               ...                        
30103    [4.801846873167803, 6.019274584642149, 4.80184...
30104    [4.801846873167803, 6.019274584642149, 4.80184...
30105    [5.525682027999934, 2.1785802706471813, 3.0797...
30106    [5.525682027999934, 2.1785802706471813, 3.0797...
30107    [5.525682027999934, 2.1785802706471813, 3.0797...
Length: 30108, dtype: object

In [54]:
def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dict
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [104]:
replaced_closeness_scores = df.text.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))

In [105]:
replaced_closeness_scores

0        [1.111805915185031, 1.0710539898330051, 1.0307...
1        [1.0855379311900761, 1.0326115459561998, 1.061...
2        [-1.0169813665612542, 0.9833075433255353, -1.2...
3        [1.0855379311900761, 1.0909515282991626, 1.084...
4        [-1.0651556483761053, 1.008110958426347, -1.40...
                               ...                        
30103    [1.099641247462443, 1.0704840662015869, 1.0996...
30104    [1.099641247462443, 1.0704840662015869, 1.0996...
30105    [1.0379936939510797, 1.0539324818220464, 1.090...
30106    [1.0379936939510797, 1.0539324818220464, 1.090...
30107    [1.0379936939510797, 1.0539324818220464, 1.090...
Name: text, Length: 30108, dtype: object

In [58]:
df["rate"]=[1 for i in range(len(df))]

In [106]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, df.less_toxic]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence']
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')

In [107]:
replacement_df.prediction.value_counts()

1    27899
0     2209
Name: prediction, dtype: int64

In [108]:
less_toxic_score=replacement_df.sentiment_rate

In [109]:
less_toxic_score

0         -50.337755
1         135.037336
2          21.515241
3          99.990358
4          -3.069074
            ...     
30103    1876.926795
30104    1876.926795
30105     128.727188
30106     128.727188
30107     128.727188
Name: sentiment_rate, Length: 30108, dtype: float64

In [110]:
df.more_toxic = df.more_toxic.apply(lambda x: ' '.join(text_cleaning(x)))
df.more_toxic = df.more_toxic.apply(lambda x: lemmatizing(x))

In [111]:
sentences = [word_tokenize(sent) for sent in df.more_toxic]
phrases = Phrases(sentences, min_count=30, progress_per=10000)
bigram = Phraser(phrases)
sentences2 = bigram[sentences]

In [112]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

w2v_model.build_vocab(sentences2, progress_per=50000)

In [113]:
w2v_model.train(sentences2, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(14630630, 61012470)

In [114]:
w2v_model.init_sims(replace=True)

C:\Users\Hector\AppData\Local\Temp/ipykernel_11228/514372312.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [115]:
w2v_model.save("word2vec.model")

In [116]:
word_vectors = Word2Vec.load("word2vec.model").wv

In [117]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

In [118]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)

[('statem', 0.9179545044898987),
 ('anonymiss', 0.9165307879447937),
 ('madchen', 0.9155290722846985),
 ('huge_faggot', 0.9146426320075989),
 ('securi', 0.9073453545570374),
 ('hi_moron', 0.902411162853241),
 ('nigger_j', 0.8992372155189514),
 ('facists_so', 0.8987661600112915),
 ('sex_sexsex', 0.8983744978904724),
 ('vandalism_rule', 0.8981286883354187)]

In [119]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

[('culus', 0.9900912642478943),
 ('coelcan', 0.9803354144096375),
 ('lifeutosigned', 0.9765209555625916),
 ('becaouse', 0.976019024848938),
 ('fcockc', 0.9752219915390015),
 ('aget', 0.9743158221244812),
 ('homun', 0.9734430313110352),
 ('fag_fag', 0.9713580012321472),
 ('drink_djathinkimacowboy', 0.971011221408844),
 ('love_juice', 0.9700945615768433)]

In [121]:
w2v_model.wv.most_similar(positive=["fuck"])

[('yourself_go', 0.846086323261261),
 ('atheist_cunt', 0.819189190864563),
 ('up_shut', 0.8134414553642273),
 ('wikipeia', 0.7855981588363647),
 ('uuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuu',
  0.7218793630599976),
 ('doit', 0.6687103509902954),
 ('wekipedia', 0.667383074760437),
 ('dustin', 0.6628047227859497),
 ('shinobis', 0.6591443419456482),
 ('carmegenon', 0.6563659906387329)]

In [122]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [123]:
words = pd.DataFrame(word_vectors.index_to_key)


In [124]:
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [125]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [126]:
sentiment_map = words
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

In [127]:
sentences3=[]
for i in sentences2:
    sentences3.append(" ".join(i))

In [128]:
df["more_toxic"]=sentences3

In [129]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x.split(), norm=None)
tfidf.fit(df.more_toxic)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(df.more_toxic)

C:\Users\Hector\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Hector\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [132]:
replaced_closeness_scores = df.text.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))

In [133]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, df.more_toxic]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence']
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')

In [134]:
more_toxic_score=replacement_df.sentiment_rate

In [160]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, df.less_toxic]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence']
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')

In [161]:
less_toxic_score2=replacement_df.sentiment_rate

In [170]:
df2=pd.read_csv("../../../jigsaw-toxic-severity-rating/validation_data.csv")

In [175]:
df2.drop("worker", axis=1, inplace=True)

In [177]:
append=pd.DataFrame(df2.more_toxic)

In [181]:
append.columns=["less_toxic"]

In [178]:
df2.drop("more_toxic", axis=1, inplace=True)

In [193]:
dff=pd.concat([df2, append])

In [194]:
dff.columns=["more_toxic"]

In [195]:
dff["index"]=range(0,len(dff))

In [196]:
dff.set_index("index")

,more_toxic
index,
0,This article sucks \n\nwoo woo wooooooo
1,"""And yes, people should recognize that but the..."
2,"Western Media?\n\nYup, because every crime in..."
3,And you removed it! You numbskull! I don't car...
4,smelly vagina \n\nBluerasberry why don't you ...
...,...
60211,get out my large penis
60212,get out my large penis
60213,Piss off you slant eyed-gook


In [197]:
dff.more_toxic = dff.more_toxic.apply(lambda x: ' '.join(text_cleaning(x)))
dff.more_toxic = dff.more_toxic.apply(lambda x: lemmatizing(x))

In [198]:
sentences = [word_tokenize(sent) for sent in dff.more_toxic]
phrases = Phrases(sentences, min_count=30, progress_per=10000)
bigram = Phraser(phrases)
sentences2 = bigram[sentences]

In [199]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

w2v_model.build_vocab(sentences2, progress_per=50000)

In [200]:
w2v_model.train(sentences2, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(31871528, 126090420)

In [201]:
w2v_model.init_sims(replace=True)

C:\Users\Hector\AppData\Local\Temp/ipykernel_11228/514372312.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [202]:
w2v_model.save("word2vec.model")

In [203]:
word_vectors = Word2Vec.load("word2vec.model").wv

In [204]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

In [205]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)

[('super_gay', 0.9195958971977234),
 ('persyour', 0.9188180565834045),
 ('onal', 0.9170994162559509),
 ('cock_suckersyou', 0.9081459641456604),
 ('shit_shit', 0.9073291420936584),
 ('watch_jealouslyfavonian', 0.9073113203048706),
 ('wierddd', 0.9072612524032593),
 ('drink_djathinkimacowboy', 0.9069275259971619),
 ('love_cock', 0.906854510307312),
 ('vomit_vomit', 0.9065504670143127)]

In [206]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

[('shitsack', 0.974092960357666),
 ('pensis', 0.9728512167930603),
 ('haaaaaaaaaaaaz', 0.9714035987854004),
 ('suggestbot', 0.9703049063682556),
 ('derivation', 0.9683917164802551),
 ('jewish_ancestryfuck', 0.9642908573150635),
 ('ancestryfuck_off', 0.9635195136070251),
 ('chester_marcolfuck', 0.9631128311157227),
 ('mexican_suck', 0.9628555178642273),
 ('sherrif', 0.9628304839134216)]

In [207]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [208]:
words = pd.DataFrame(word_vectors.index_to_key)


In [209]:
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [211]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [212]:
sentiment_map = words
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

In [213]:
sentences3=[]
for i in sentences2:
    sentences3.append(" ".join(i))

In [215]:
dff["more_toxic"]=sentences3

In [216]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x.split(), norm=None)
tfidf.fit(df.more_toxic)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(df.more_toxic)

C:\Users\Hector\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Hector\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [223]:
dff.set_index(["index"], inplace=True)

In [224]:
dff.columns=["text"]

In [226]:
replaced_closeness_scores = dff.text.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))

In [228]:
dff["closeness_scores"]=replaced_closeness_scores

In [236]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x.split(), norm=None)
tfidf.fit(dff.text)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(dff.text)

C:\Users\Hector\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Hector\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [237]:
replaced_tfidf_scores = dff.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)#this step takes around 3-4 minutes minutes to calculate

In [238]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, dff.text]).T

In [239]:
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence']


In [240]:
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)


In [254]:
replacement_df["sentiment_rate"]

index
0         47.933236
1       -192.095769
2        -77.389680
3        -83.262213
4        -42.769537
            ...    
60211    -21.002575
60212    -21.002575
60213      0.322755
60214      0.322755
60215      0.322755
Name: sentiment_rate, Length: 60216, dtype: float64

In [255]:
less_toxic_score=replacement_df.sentiment_rate[0:30108]

In [256]:
more_toxic_score=replacement_df.sentiment_rate[30108:]

In [257]:
more_toxic_score

index
30108    -30.030800
30109   -154.168853
30110   -606.202330
30111    -75.574594
30112    -27.320759
            ...    
60211    -21.002575
60212    -21.002575
60213      0.322755
60214      0.322755
60215      0.322755
Name: sentiment_rate, Length: 30108, dtype: float64

In [244]:
vd = pd.read_csv("../../../jigsaw-toxic-severity-rating/validation_data.csv")

In [245]:
vd

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"
...,...,...,...
30103,461,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30104,527,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30105,352,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook
30106,311,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook


In [262]:
vd["less_toxic_score"]=replacement_df.sentiment_rate[0:30108]

In [266]:
vd["more_toxic_score"]=mts

In [270]:
type(more_toxic_score)

pandas.core.series.Series

In [271]:
type(less_toxic_score)

pandas.core.series.Series

In [272]:
less_toxic_score

index
0          47.933236
1        -192.095769
2         -77.389680
3         -83.262213
4         -42.769537
            ...     
30103   -1874.818409
30104   -1874.818409
30105    -127.437359
30106    -127.437359
30107    -127.437359
Name: sentiment_rate, Length: 30108, dtype: float64

In [277]:
mts=mts.tolist()

In [280]:
vd["more_toxic_score"]=mts

In [281]:
vd

,worker,less_toxic,more_toxic,less_toxic_score,more_toxic_score,more_score
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,47.933236,-30.030800,-30.030800
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,-192.095769,-154.168853,-154.168853
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",-77.389680,-606.202330,-606.202330
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,-83.262213,-75.574594,-75.574594
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",-42.769537,-27.320759,-27.320759
...,...,...,...,...,...,...
30103,461,I'm sorry. I'm not an admin. I will give you t...,get out my large penis,-1874.818409,-21.002575,-21.002575
30104,527,I'm sorry. I'm not an admin. I will give you t...,get out my large penis,-1874.818409,-21.002575,-21.002575
30105,352,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook,-127.437359,0.322755,0.322755
30106,311,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook,-127.437359,0.322755,0.322755


In [282]:
scores=[]
for index, row in vd.iterrows():
    if row["less_toxic_score"]>=row["more_toxic_score"]:
        scores.append(1)
    else:
        scores.append(0)

In [283]:
vd["scores"]=scores

In [284]:
vd

,worker,less_toxic,more_toxic,less_toxic_score,more_toxic_score,more_score,scores
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,47.933236,-30.030800,-30.030800,1
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,-192.095769,-154.168853,-154.168853,0
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",-77.389680,-606.202330,-606.202330,1
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,-83.262213,-75.574594,-75.574594,0
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",-42.769537,-27.320759,-27.320759,0
...,...,...,...,...,...,...,...
30103,461,I'm sorry. I'm not an admin. I will give you t...,get out my large penis,-1874.818409,-21.002575,-21.002575,0
30104,527,I'm sorry. I'm not an admin. I will give you t...,get out my large penis,-1874.818409,-21.002575,-21.002575,0
30105,352,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook,-127.437359,0.322755,0.322755,0
30106,311,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook,-127.437359,0.322755,0.322755,0


In [285]:
vd["scores"].value_counts(normalize=True)

0    0.5464
1    0.4536
Name: scores, dtype: float64

In [152]:
vd = vd[vd.less_toxic_score != 0]

In [153]:
vd = vd[vd.more_toxic_score != 0]

In [154]:
vd["scores"].value_counts(normalize=True)

1    0.936252
0    0.063748
Name: scores, dtype: float64

In [155]:
len(vd)

30103